In [42]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
sns.set(style='darkgrid')
import pickle   # For saving and loading machine learning models
from sklearn.model_selection import train_test_split  # For splitting the data into training and testing
from imblearn.over_sampling import SMOTE  # For balancing the data


In [43]:
data = pd.read_csv(r"C:\Users\ALVIN\INX_Employee_Performance\data\processed\final_encoded_employee_data.csv")

In [44]:
data

Age  EmpHourlyRate  EmpLastSalaryHikePercent  \
0    -0.541458      -0.543569                 -0.932543   
1     1.109888      -1.187042                 -0.932543   
2     0.339260      -0.890055                  1.543296   
3     0.449349       0.347393                  0.044626   
4     2.541054       0.891870                 -0.259098   
...        ...            ...                       ...   
1195 -1.091906       0.248397                  1.324369   
1196  0.008991       0.693878                  0.598923   
1197  1.440157       0.396891                 -1.309231   
1198 -0.321278      -0.989050                 -0.259098   
1199 -1.422176      -0.048590                 -0.259098   

      TotalWorkExperienceInYears  YearsSinceLastPromotion  \
0                       0.152962                -1.011363   
1                       1.086134                -0.096723   
2                       1.086134                -0.096723   
3                       1.278838                 1.812554   
4                       0.152962                 0.438307   
...                          ...                      ...   
1195                   -0.499315                -1.011363   
1196                   -0.984890                -1.011363   
1197                    1.086134                 0.817917   
1198                    0.015416                 1.732557   
1199                   -0.984890                 0.438307   

      ExperienceYearsAtThisCompany  ExperienceYearsInCurrentRole  \
0                         0.795728                      0.771213   
1                         0.366905                      0.771213   
2                         1.531691                      2.465670   
3                         1.698147                      0.488804   
4                        -0.953858                     -0.640834   
...                            ...                           ...   
1195                      0.187094                      0.206394   
1196                     -1.499849                     -1.205653   
1197                      1.666462                      1.053623   
1198                      0.525509                      0.771213   
1199                     -0.953858                     -0.640834   

      YearsWithCurrManager  EmpJobRole  Gender_Male  ...  \
0                 1.121693        18.0          1.0  ...   
1                 0.834907        18.0          1.0  ...   
2                 2.268839        18.0          1.0  ...   
3                 0.548120        11.0          1.0  ...   
4                -0.599025        18.0          1.0  ...   
...                    ...         ...          ...  ...   
1195             -0.025452        18.0          0.0  ...   
1196             -1.172598        12.0          1.0  ...   
1197              1.121693        12.0          1.0  ...   
1198              0.834907         5.0          0.0  ...   
1199             -1.172598        18.0          0.0  ...   

      EducationBackground_Technical Degree  EmpDepartment_Development  \
0                                      0.0                        0.0   
1                                      0.0                        0.0   
2                                      0.0                        0.0   
3                                      0.0                        0.0   
4                                      0.0                        0.0   
...                                    ...                        ...   
1195                                   0.0                        0.0   
1196                                   0.0                        1.0   
1197                                   0.0                        1.0   
1198                                   0.0                        0.0   
1199                                   0.0                        0.0   

      EmpDepartment_Finance  EmpDepartment_Human Resources  \
0                       0.0                            0.0   
1                       0.0   

In [45]:
# Separate features and target
X = data.drop('PerformanceRating', axis=1)
y = data['PerformanceRating']

In [46]:
# Spliting the data into train and test
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=33)

In [49]:
import time
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_sample_weight

In [61]:
models = {}  # Dictionary to store all trained models


### 1. Logistic Regression

In [62]:
from sklearn.model_selection import GridSearchCV

log_param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['liblinear', 'lbfgs']
}

log_grid = GridSearchCV(LogisticRegression(class_weight='balanced', random_state=42),
                        log_param_grid, cv=3, scoring='accuracy', n_jobs=-1)

log_grid.fit(x_train, y_train)
print("Best Params:", log_grid.best_params_)

# Final model
best_log_model = LogisticRegression(**log_grid.best_params_, class_weight='balanced', random_state=42)
best_log_model.fit(x_train, y_train)

Best Params: {'C': 0.01, 'solver': 'liblinear'}


LogisticRegression(C=0.01, class_weight='balanced', random_state=42,
                   solver='liblinear')

### 2. Decision Tree

In [63]:
# Define Hyperparameters for Decision Tree
dt_param_grid = {
    'max_depth': [3, 5, 10, None],
    'criterion': ['gini', 'entropy']
}
# Train Decision Tree with GridSearchCV
dt_grid = GridSearchCV(DecisionTreeClassifier(class_weight='balanced', random_state=42),
                       dt_param_grid, cv=3, scoring='accuracy', n_jobs=-1, verbose=2)
dt_grid.fit(x_train, y_train)
print("Best Parameters for Decision Tree:", dt_grid.best_params_)
# Train Final Decision Tree Model
best_dt_model = DecisionTreeClassifier(**dt_grid.best_params_, class_weight='balanced', random_state=42)
best_dt_model.fit(x_train, y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
Best Parameters for Decision Tree: {'criterion': 'entropy', 'max_depth': None}


DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
                       random_state=42)

### 3. Random Forest

In [64]:
# Define Hyperparameters for Random Forest
rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20]
}
# Train Random Forest with GridSearchCV
rf_grid = GridSearchCV(RandomForestClassifier(class_weight='balanced', random_state=42),
                       rf_param_grid, cv=3, scoring='accuracy', n_jobs=-1, verbose=2)
rf_grid.fit(x_train, y_train)
print("Best Parameters for Random Forest:", rf_grid.best_params_)
# Train Final Random Forest Model
best_rf_model = RandomForestClassifier(**rf_grid.best_params_, class_weight='balanced', random_state=42)
best_rf_model.fit(x_train, y_train)

Fitting 3 folds for each of 9 candidates, totalling 27 fits
Best Parameters for Random Forest: {'max_depth': 20, 'n_estimators': 50}


RandomForestClassifier(class_weight='balanced', max_depth=20, n_estimators=50,
                       random_state=42)

### 4. Gradient Boosting

In [65]:
# Define Hyperparameters for Gradient Boosting
gb_param_grid = {
    'n_estimators': [50, 100],
    'learning_rate': [0.01, 0.1]
}
# Train Gradient Boosting with GridSearchCV
gb_grid = GridSearchCV(GradientBoostingClassifier(random_state=42),
                       gb_param_grid, cv=3, scoring='accuracy', n_jobs=-1, verbose=2)
gb_grid.fit(x_train, y_train)
print("Best Parameters for Gradient Boosting:", gb_grid.best_params_)
# Train Final Gradient Boosting Model
best_gb_model = GradientBoostingClassifier(**gb_grid.best_params_, random_state=42)
best_gb_model.fit(x_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
Best Parameters for Gradient Boosting: {'learning_rate': 0.01, 'n_estimators': 100}


GradientBoostingClassifier(learning_rate=0.01, random_state=42)

### 5.XGBoost

In [67]:
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
import joblib

# Step 1: Create the models dictionary
models = {}

# Step 2: Label Encode target
le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)
y_test_encoded = le.transform(y_test)

# Step 3: Train XGBoost
xgb_model = XGBClassifier(
    objective="multi:softprob",
    num_class=3,
    eval_metric="mlogloss",
    use_label_encoder=False,
    random_state=42
)

xgb_model.fit(x_train, y_train_encoded)
models["XGBoost"] = xgb_model


### 6.SVM

In [68]:
from sklearn.svm import SVC
# Define Hyperparameters for SVM
svm_param_grid = {
    'C': [0.1, 1],  # Regularization parameter
    'kernel': ['linear'],  # Kernel types
    'gamma': ['scale', 'auto']  # Kernel coefficient
}

# Train SVM with GridSearchCV
svm_grid = GridSearchCV(SVC(random_state=42),
                        svm_param_grid, cv=3, scoring='accuracy', n_jobs=-1, verbose=2)
svm_grid.fit(x_train, y_train)
print("Best Parameters for SVM:", svm_grid.best_params_)
# Train Final SVM Model
best_svm_model = SVC(**svm_grid.best_params_, random_state=42)
best_svm_model.fit(x_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
Best Parameters for SVM: {'C': 1, 'gamma': 'scale', 'kernel': 'linear'}


SVC(C=1, kernel='linear', random_state=42)

In [69]:
import joblib

# Create dictionary and add each trained model
models = {}

# Logistic Regression
models["LogisticRegression"] = best_log_model

# Decision Tree
models["DecisionTree"] = best_dt_model

# Random Forest
models["RandomForest"] = best_rf_model

# Gradient Boosting
models["GradientBoosting"] = best_gb_model

# SVM
models["SVM"] = best_svm_model

# XGBoost (trained with label-encoded y)
models["XGBoost"] = xgb_model

# Save all models to one file
joblib.dump(models, "../models/all_models.pkl")

# Save the label encoder used for XGBoost
joblib.dump(le, "../models/label_encoder.pkl")

print("✅ All models saved to 'models/all_models.pkl'")
print("✅ LabelEncoder saved to 'models/label_encoder.pkl'")


✅ All models saved to 'models/all_models.pkl'
✅ LabelEncoder saved to 'models/label_encoder.pkl'
